In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/eda-missing-values-tsne-clustering-class-imb/weights_1
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/weights_2
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/__results__.html
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/features_names
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/submission.csv
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/__resultx__.html
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/weights_0
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/__notebook__.ipynb
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/__output__.json
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/features_imputation
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/custom.css
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/__results___files/__results___80_0.png
/kaggle/input/eda-missing-values-tsne-clustering-class-imb/__results___files/__results___15_0

In [2]:
import random
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
from sklearn.model_selection import train_test_split
import lightgbm as lgbm
import pickle

In [3]:
def load_pickle(fich):
    with open(fich,"rb") as f :
        contenu = pickle.load(f)
    return contenu

In [4]:
path = "../input/eda-missing-values-tsne-clustering-class-imb"

In [5]:
SEED = 1111

In [6]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [7]:
print('load train start')
try:
    train = pd.read_parquet('train.parquet')
except:
    train = pd.read_csv('../input/jane-street-market-prediction/train.csv', index_col='ts_id')
    train.to_parquet('train.parquet')
print('load train end')

load train start


/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


load train end


In [8]:
# train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)

train['action'] = ((train['resp'].values) > 0).astype(int)


#features = [c for c in train.columns if "feature" in c]
features = load_pickle(os.path.join(path,"features_names"))

#f_mean = np.mean(train[features[1:]].values,axis=0)
f_mean = load_pickle(os.path.join(path,"features_imputation"))

with open("imputation","wb") as f :
    pickle.dump(f_mean,f)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X_train = train.loc[:, features]
#y_train = (train.loc[:, 'action'])

y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

# multimodel

In [9]:
# modeling step 
params={"num_leaves":300,
       "max_bin":450,
       "feature_fraction":0.52,
       "bagging_fraction":0.52,
       "objective":"binary",
       "learning_rate":0.05,
       "boosting_type":"gbdt",
       "metric":"auc"
       }
models = [] # list of model , we will train 
for i in tqdm(range(y_train.shape[1])):
    xtr,xval,ytr,yval = train_test_split(X_train ,y_train[:,i],test_size=0.2,stratify=y_train[:,i])
   
    d_train = lgbm.Dataset(xtr,label=ytr)
    d_eval = lgbm.Dataset(xval,label=yval,reference=d_train)
    clf = lgbm.train(params,d_train,valid_sets=[d_train,d_eval],num_boost_round=1500,\
                    early_stopping_rounds=50,verbose_eval=50)
    nom_fich = "weights_target_" + resp_cols[i]
    clf.save_model(nom_fich)
    models.append(clf)

  0%|          | 0/5 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 634178, number of negative: 622954
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.785402 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45450
[LightGBM] [Info] Number of data points in the train set: 1257132, number of used features: 101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504464 -> initscore=0.017857
[LightGBM] [Info] Start training from score 0.017857
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.619652	valid_1's auc: 0.585666
[100]	training's auc: 0.649792	valid_1's auc: 0.59848
[150]	training's auc: 0.672489	valid_1's auc: 0.603955
[200]	training's auc: 0.692672	valid_1's auc: 0.60645
[250]	training's auc: 0.712043	valid_1's auc: 0.608578
[300]	training's auc: 0.729683	valid_1's auc: 0.61016
[350]	training's auc: 0.745812	valid_1's auc: 0.612117
[400]	training's auc: 0.760207	valid_1's auc: 0.613896
[4

 20%|██        | 1/5 [10:56<43:44, 656.11s/it]

[LightGBM] [Info] Number of positive: 636450, number of negative: 620682
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.760073 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45450
[LightGBM] [Info] Number of data points in the train set: 1257132, number of used features: 101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506271 -> initscore=0.025087
[LightGBM] [Info] Start training from score 0.025087
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.615277	valid_1's auc: 0.578221
[100]	training's auc: 0.648773	valid_1's auc: 0.593661
[150]	training's auc: 0.675229	valid_1's auc: 0.600883
[200]	training's auc: 0.697686	valid_1's auc: 0.606715
[250]	training's auc: 0.718143	valid_1's auc: 0.609972
[300]	training's auc: 0.7388	valid_1's auc: 0.613239
[350]	training's auc: 0.75481	valid_1's auc: 0.615215
[400]	training's auc: 0.77092	valid_1's auc: 0.617561
[45

 40%|████      | 2/5 [21:29<32:07, 642.47s/it]

...

 60%|██████    | 3/5 [32:02<21:16, 638.18s/it]

[LightGBM] [Info] Number of positive: 631460, number of negative: 625672
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.926924 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45450
[LightGBM] [Info] Number of data points in the train set: 1257132, number of used features: 101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502302 -> initscore=0.009208
[LightGBM] [Info] Start training from score 0.009208
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.637307	valid_1's auc: 0.605131
[100]	training's auc: 0.682137	valid_1's auc: 0.630011
[150]	training's auc: 0.716355	valid_1's auc: 0.644116
[200]	training's auc: 0.745205	valid_1's auc: 0.650989
[250]	training's auc: 0.770642	valid_1's auc: 0.659239
[300]	training's auc: 0.793221	valid_1's auc: 0.664381
[350]	training's auc: 0.812899	valid_1's auc: 0.668416
[400]	training's auc: 0.829237	valid_1's auc: 0.671315

 80%|████████  | 4/5 [42:01<10:22, 622.94s/it]

[LightGBM] [Info] Number of positive: 631877, number of negative: 625255
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.788870 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45450
[LightGBM] [Info] Number of data points in the train set: 1257132, number of used features: 101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502634 -> initscore=0.010535
[LightGBM] [Info] Start training from score 0.010535
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.648785	valid_1's auc: 0.616519
[100]	training's auc: 0.696328	valid_1's auc: 0.642136
[150]	training's auc: 0.731134	valid_1's auc: 0.655362
[200]	training's auc: 0.761306	valid_1's auc: 0.666646
[250]	training's auc: 0.784247	valid_1's auc: 0.672545
[300]	training's auc: 0.804519	valid_1's auc: 0.677756
[350]	training's auc: 0.822453	valid_1's auc: 0.684548
[400]	training's auc: 0.839964	valid_1's auc: 0.690749

100%|██████████| 5/5 [52:06<00:00, 625.33s/it]


# single_model

In [10]:
params={"num_leaves":300,
       "max_bin":450,
       "feature_fraction":0.52,
       "bagging_fraction":0.52,
       "objective":"binary",
       "learning_rate":0.05,
       "boosting_type":"gbdt",
       "metric":"auc"
       }
sing_model = []
for i in range(3):
  
    clf = lgbm.Booster(model_file=os.path.join(path,"weights_{}".format(i)))
    sing_model.append(clf)

In [11]:
f = np.median
th = 0.5000
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt.sum()):
            x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean
        # multimodel
        pred1 = np.mean(np.stack([model.predict(x_tt) for model in models]),axis=0).T
        # single_model
        pred2 = np.mean(np.stack([model.predict(x_tt) for model in sing_model]),axis=0)
        
        pred = 0.5 * pred1 + 0.5 * pred2
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

15219it [05:12, 48.69it/s]
